Készítsünk sorszámozott listát a szálláshelyek adatairól! 

a. A lista legyen a szálláshely típusa szerint, azon belül hely szerint, majd a neve szerint növekvően rendezve! 

b. A számozás típusonként kezdődjön újra!

In [ ]:
select *,ROW_NUMBER() over(PARTITION by tipus order by tipus,hely,szallas_nev)
 from  Szallashely

3

Készítsük listát, amely megjeleníti az ügyfelek azonosítóját, nevét és a foglalásainak kezdő- és befejező dátumát, valamint azt, hogy a foglalás összesen hány főre történt! 

a. Egy új oszlopban jelenítsük meg azt is, hogy az ügyfél előző három foglalása összesen hány főre történt!

In [ ]:
Select u.[LOGIN], u.NEV, f.METTOL, f.MEDDIG,f.FELNOTT_SZAM + f.GYERMEK_SZAM,
sum(f.FELNOTT_SZAM + f.GYERMEK_SZAM) OVER (PARTITION by u.LOGIN ORDER by f.mettol ROWS BETWEEN 3 Preceding and 1 Preceding)
from Foglalas f join  Ugyfel u on f.UGYFEL_FK=u.[LOGIN]

6

<span style="color: rgb(18, 38, 44); font-family: Arial; font-size: 15px; background-color: rgb(239, 249, 252);">Alakítsa át a melléklet 1. pontjában lévő lekérdezést úgy, hogy JOIN helyett CROSS APPLY-t alkalmaz!</span>  
<span style="color: rgb(18, 38, 44); font-family: Arial; font-size: 15px; background-color: rgb(239, 249, 252);">a. Válaszként adja meg az átalakított lekérdezés kódját!</span>

In [ ]:
SELECT rt.TERMEKKOD, 
       AVG(rt.EGYSEGAR) AS 'Átlagos ár'
FROM Rendeles_tetel rt 
CROSS APPLY
(
    Select * from Rendeles r
     where rt.SORSZAM = r.SORSZAM
)r
GROUP BY rt.TERMEKKOD